In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules

### Data PRep

In [4]:
data = pd.read_csv('Data/orders_train_instacart.csv')


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   order_id                1384617 non-null  int64 
 1   product_id              1384617 non-null  int64 
 2   add_to_cart_order       1384617 non-null  int64 
 3   reordered               1384617 non-null  int64 
 4   user_id                 1384617 non-null  int64 
 5   eval_set                1384617 non-null  object
 6   order_number            1384617 non-null  int64 
 7   order_dow               1384617 non-null  int64 
 8   order_hour_of_day       1384617 non-null  int64 
 9   days_since_prior_order  1384617 non-null  int64 
 10  product_name            1384617 non-null  object
 11  aisle_id                1384617 non-null  int64 
 12  department_id           1384617 non-null  int64 
 13  aisle                   1384617 non-null  object
 14  department        

In [4]:
df_item = data[['order_id','product_name']].copy()
df_item.rename(columns={'order_id':'order','product_name':'items'},inplace=True)
df_item['temp']=1
df = df_item.groupby(['order','items'])['temp'].sum().unstack().fillna(0)
def myencoder(i):
    if i <= 0:
        return 0
    elif i>=1:
        return 1
df = df.applymap(myencoder)
df[df>1] = 1
df

items,#2 Coffee Filters,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Superfruits Greek Yogurt,0% Greek Strained Yogurt,1 Apple + 1 Pear Fruit Bar,1 Liter,1 Ply Paper Towels,1 Razor Handle and 2 Freesia Scented Razor Refills Premium BladeRazor System,1% Chocolate Milk,...,for Tots Apple Juice,for Women Maximum Absorbency L Underwear,gel hand wash sea minerals,of Norwich Original English Mustard Powder Double Superfine,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise Dressing
order,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 1. Market BAsket ANalysis

### 1. Apriori Algorithm

In [5]:
freq_itemsets = apriori(df,min_support=0.005,use_colnames=True)
freq_itemsets

,support,itemsets
0,0.008752,(100% Raw Coconut Water)
1,0.009266,(100% Recycled Paper Towels)
2,0.015444,(100% Whole Wheat Bread)
3,0.012098,(2% Reduced Fat Milk)
4,0.018018,(Apple Honeycrisp Organic)
...,...,...
389,0.005920,"(Organic Strawberries, Organic Zucchini)"
390,0.005663,"(Organic Strawberries, Seedless Red Grapes)"
391,0.005148,"(Organic Yellow Onion, Organic Zucchini)"
392,0.005405,"(Organic Raspberries, Organic Hass Avocado, Ba..."


### 2. Get Rules

In [6]:
rules = association_rules(freq_itemsets,metric='lift',min_threshold=1)
rules_df=rules.sort_values(by='confidence',ascending=False)

In [7]:
rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
238,"(Organic Raspberries, Organic Hass Avocado)",(Bag of Organic Bananas),0.007979,0.119949,0.005405,0.677419,5.647584,0.004448,2.728160
244,"(Organic Hass Avocado, Organic Strawberries)",(Bag of Organic Bananas),0.010296,0.119949,0.005920,0.575000,4.793723,0.004685,2.070709
0,(Apple Honeycrisp Organic),(Bag of Organic Bananas),0.018018,0.119949,0.007722,0.428571,3.572961,0.005561,1.540090
111,(Roma Tomato),(Banana),0.014414,0.141313,0.006178,0.428571,3.032787,0.004141,1.502703
239,"(Organic Raspberries, Bag of Organic Bananas)",(Organic Hass Avocado),0.013642,0.053024,0.005405,0.396226,7.472522,0.004682,1.568428
...,...,...,...,...,...,...,...,...,...
85,(Banana),(Organic Cilantro),0.141313,0.026255,0.005405,0.038251,1.456927,0.001695,1.012474
107,(Banana),(Original Hummus),0.141313,0.019048,0.005405,0.038251,2.008197,0.002714,1.019968
90,(Banana),(Organic Garnet Sweet Potato (Yam)),0.141313,0.022651,0.005148,0.036430,1.608296,0.001947,1.014300
98,(Banana),(Organic Red Onion),0.141313,0.032947,0.005148,0.036430,1.105704,0.000492,1.003614


In [8]:
rules_df[(rules_df['confidence']>0.16) & (rules_df['lift']>1)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
238,"(Organic Raspberries, Organic Hass Avocado)",(Bag of Organic Bananas),0.007979,0.119949,0.005405,0.677419,5.647584,0.004448,2.728160
244,"(Organic Hass Avocado, Organic Strawberries)",(Bag of Organic Bananas),0.010296,0.119949,0.005920,0.575000,4.793723,0.004685,2.070709
0,(Apple Honeycrisp Organic),(Bag of Organic Bananas),0.018018,0.119949,0.007722,0.428571,3.572961,0.005561,1.540090
111,(Roma Tomato),(Banana),0.014414,0.141313,0.006178,0.428571,3.032787,0.004141,1.502703
239,"(Organic Raspberries, Bag of Organic Bananas)",(Organic Hass Avocado),0.013642,0.053024,0.005405,0.396226,7.472522,0.004682,1.568428
...,...,...,...,...,...,...,...,...,...
224,(Organic Yellow Onion),(Organic Lemon),0.031145,0.029601,0.005148,0.165289,5.583902,0.004226,1.162557
37,(Bag of Organic Bananas),(Organic Hass Avocado),0.119949,0.053024,0.019820,0.165236,3.116224,0.013460,1.134423
82,(Organic Blueberries),(Banana),0.039125,0.141313,0.006435,0.164474,1.163898,0.000906,1.027720
229,(Organic Red Onion),(Organic Strawberries),0.032947,0.085714,0.005405,0.164063,1.914063,0.002581,1.093725


In [9]:
rules_df['lift'] = round(rules_df['lift'],2)

rules_df['antecedents']=rules_df['antecedents'].astype(str).str.replace('frozenset',"")
rules_df['consequents']=rules_df['consequents'].astype(str).str.replace('frozenset',"")

In [10]:
top_rules_df= rules_df.sort_values('lift',ascending=False).head(100)

In [12]:
rules_df.to_csv('rules.csv')

In [13]:
rules_df.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
238,"({'Organic Raspberries', 'Organic Hass Avocado'})",({'Bag of Organic Bananas'}),0.007979,0.119949,0.005405,0.677419,5.65,0.004448,2.728160
244,"({'Organic Hass Avocado', 'Organic Strawberrie...",({'Bag of Organic Bananas'}),0.010296,0.119949,0.005920,0.575000,4.79,0.004685,2.070709
0,({'Apple Honeycrisp Organic'}),({'Bag of Organic Bananas'}),0.018018,0.119949,0.007722,0.428571,3.57,0.005561,1.540090
111,({'Roma Tomato'}),({'Banana'}),0.014414,0.141313,0.006178,0.428571,3.03,0.004141,1.502703
239,"({'Organic Raspberries', 'Bag of Organic Banan...",({'Organic Hass Avocado'}),0.013642,0.053024,0.005405,0.396226,7.47,0.004682,1.568428


# 2. Visualization for Rule's Network 

In [494]:
import plotly.express as px


In [496]:
got_net = Network(
    height="850px", 
    width="100%", 
    bgcolor="black", 
    font_color="white", 
    notebook=True # here
)

# set the physics layout of the network
got_net.barnes_hut()
got_data = top_rules_df
got_net.toggle_hide_edges_on_drag(False)
sources = got_data['consequents']
targets = got_data['antecedents']
weights = got_data['lift']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=f"<h4>{src}</h4>")
    got_net.add_node(dst, dst, title=f"<h4>{dst}</h4>")
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += '<b> Neighbors</b>:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])
got_net.show_buttons(filter_=['nodes'])

got_net.show('Rules_network.html')
